## Sistemas de recomendação baseados em Filtragem colaborativa de fatoração de matriz e Rede Neural em Keras.

In [ ]:
# Ignorando alguns conjunto de Erros
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

# Pacotes de visualização e Manipulação de Dados
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns
 
#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline  
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#Secção relacionada a criação do modelo
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#pré-processamento.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , merge
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau

from keras.layers.merge import dot
from keras.models import Model

# Biblioteca específica para deeplearning.
from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
from IPython.display import SVG

#Tratamento de Imagem
import cv2                  
import numpy as np  
from tqdm import tqdm
import os                   
from random import shuffle  
from zipfile import ZipFile
from PIL import Image

#modelo pré-treinado VGG-16
from keras.applications.vgg16 import VGG16

## 1.2 ) Lendo o arquivo CSV 

In [ ]:
train=pd.read_csv(r'../input/movielens100k/ratings.csv')

Observe que este arquivo contém as classificações dadas para um conjunto de usuários em diferentes filmes. Ao todo, ele contém classificações totais de 100K; para ser exato 1000004.

In [ ]:
df=train.copy()

In [ ]:
df.head()

## 1.3 ) Explorando o Banco de Dados em CSV

In [ ]:
df['userId'].unique()

In [ ]:
len(df['userId'].unique())

* Observe que, no total, temos 671 usuários cujo ID de usuário varia de 1 a> 671.

In [ ]:
df['movieId'].unique()

In [ ]:
len(df['movieId'].unique())

Da mesma forma, temos 9066 filmes exclusivos. Observe também que, conforme informado, cada usuário votou em pelo menos em 20 filmes. Veremos que a matriz é gigante.

# # #### Observe que, para 671 usuários e 9066 filmes, podemos ter no máximo 671 * 9066 = 6083286 avaliações. Mas observe que temos apenas 100004 avaliações conosco. Portanto, a matriz de utilidade tem apenas cerca de 1**,6% dos valores totais. Assim, pode-se concluir que é bastante esparso. Isso limita o uso de alguns algoritmos. Portanto, criaremos embeddings para eles mais tarde.

In [ ]:
df['userId'].isnull().sum()

In [ ]:
df['rating'].isnull().sum()

In [ ]:
df['movieId'].isnull().sum()

Isso confirma que nenhuma das colunas possui qualquer valor NULL ou Nan.

In [ ]:
df['rating'].min() # minimum rating

In [ ]:
df['rating'].max() # maximum rating

1. <a id="content2"></a>
1. ## 2 ) Preparando os Dados

* * ## 2.1 ) As colunas passam a ser o tipo "categoria" Pandas

In [ ]:
df.userId = df.userId.astype('category').cat.codes.values
df.movieId = df.movieId.astype('category').cat.codes.values

In [ ]:
df['userId'].value_counts(ascending=True)

In [ ]:
df['movieId'].unique()

## 2.2 ) Criando uma Matriz de Utilidade

In [ ]:
# criando matriz de utilidade.
index=list(df['userId'].unique())
columns=list(df['movieId'].unique())
index=sorted(index)
columns=sorted(columns)
 
util_df=pd.pivot_table(data=df,values='rating',index='userId',columns='movieId')
# Nan implica que o usuário não classificou o filme correspondente.

In [ ]:
util_df

#### QUEBRANDO -

1) Esta é a matriz de utilidade; para cada um dos 671 usuários dispostos em linha; cada coluna mostra a avaliação do filme dada por um determinado usuário.

2) Observe que a maioria da matriz é preenchida com 'Nan', o que mostra que a maioria dos filmes não é classificada por muitos usuários.

3) Para cada par filme-usuário, se a entrada NÃO for 'Nan', o valor indica a classificação dada pelo usuário ao filme correspondente.

4) Por enquanto, irei preencher o valor 'Nan' com o valor '0'. Mas observe que isso é apenas indicativo, um 0 significa SEM AVALIAÇÃO e não significa que o usuário classificou 0 para aquele filme. Não representa nenhuma classificação.

In [ ]:
util_df.fillna(0)

## 2.3 ) Criação de conjuntos de treinamento e validação.

In [ ]:
# x_train,x_test,y_train,y_test=train_test_split(df[['userId','movieId']],df[['rating']],test_size=0.20,random_state=42)
users = df.userId.unique()
movies = df.movieId.unique()

userid2idx = {o:i for i,o in enumerate(users)}
movieid2idx = {o:i for i,o in enumerate(movies)}

In [ ]:
df['userId'] = df['userId'].apply(lambda x: userid2idx[x])
df['movieId'] = df['movieId'].apply(lambda x: movieid2idx[x])
split = np.random.rand(len(df)) < 0.8
train = df[split]
valid = df[~split]
print(train.shape , valid.shape)

<a id="content3"></a>
## 3 )  Fatoração de Matriz

>> #### Aí vem a parte principal !!!

1) Agora passamos para o ponto crucial do notebook, ou seja, a fatoração de matrizes. Na facorização da matriz, basicamente dividimos uma matriz em geralmente 2 matrizes menores, cada uma com dimensões menores. essas matrizes são freqüentemente chamadas de 'Embeddings'. Podemos ter variantes de Matrix Factorizartion-> 'Low Rank MF', 'Non-Negaive MF' (NMF) e assim por diante.

2) Aqui, usei a chamada 'Fatoração de Matriz de Baixo Rank'. Criei embeddings para o usuário e também para o item; filme no nosso caso. O número de dimensões ou os chamados 'Fatores latentes' nos embeddings é um hiperparâmetro com o qual lidar nesta implementação de Filtragem Colaborativa.

## 3.1 ) Criando os Embeddings, Mesclando e Fazendo o Modelo a partir de Embeddings

In [ ]:
n_movies=len(df['movieId'].unique())
n_users=len(df['userId'].unique())
n_latent_factors=64  # hyperparamter to deal with. 

In [ ]:
user_input=Input(shape=(1,),name='user_input',dtype='int64')

In [ ]:
user_embedding=Embedding(n_users,n_latent_factors,name='user_embedding')(user_input)
#user_embedding.shape

In [ ]:
user_vec =Flatten(name='FlattenUsers')(user_embedding)
#user_vec.shape

In [ ]:
movie_input=Input(shape=(1,),name='movie_input',dtype='int64')
movie_embedding=Embedding(n_movies,n_latent_factors,name='movie_embedding')(movie_input)
movie_vec=Flatten(name='FlattenMovies')(movie_embedding)
#movie_vec

In [ ]:
sim=dot([user_vec,movie_vec],name='Simalarity-Dot-Product',axes=1)
model =keras.models.Model([user_input, movie_input],sim)
# #model.summary()
# # A summary of the model is shown below-->

#### QUEBRANDO -

1) Primeiro, precisamos criar embeddings para o usuário e também para o item ou filme. Para isso, usei a camada Embedding de keras.

2) Especifique a entrada que se espera que seja incorporada (tanto na incorporação do usuário quanto do item). Use uma camada de Embedding que espera o não de fatores latentes no embedding resultante e também o não de usuários ou itens.

3) Em seguida, pegamos o 'Produto Ponto' de ambos os embeddings usando a camada 'mesclar'. Observe que 'produto escalar' é apenas uma medida de similitude e podemos usar qualquer outro modo como 'multiplicar' ou 'simularidade de cosseno' ou 'concatenar' etc ...

4) Por último, fazemos um modelo Keras a partir dos detalhes especificados.

## 3.2 )Compilando o modelo

In [ ]:
model.compile(optimizer=Adam(lr=1e-4),loss='mse')

Observe que a métrica usada é 'Erro quadrático médio'. Nosso objetivo é minimizar o mse no conjunto de treinamento, ou seja, sobre os valores que o usuário classificou (100004 avaliações).

In [ ]:
train.shape
batch_size=128
epochs=50

## 3.3 ) Ajuste no conjunto de treinamento e validação no conjunto de validação.

In [ ]:
History = model.fit([train.userId,train.movieId],train.rating, batch_size=batch_size,
                              epochs =epochs, validation_data = ([valid.userId,valid.movieId],valid.rating),
                              verbose = 1)

<a id="content4"></a>
## 4 ) Avaliando o desempenho do modelo

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import matplotlib.pyplot as plt
plt.plot(History.history['loss'] , 'g')
plt.plot(History.history['val_loss'] , 'b')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()

<a id="content5"></a>
## 5 ) Usando Uma Neural Network

#### Agora vamos nos concentrar na outra coisa principal !!! Usando um NN para a fatoração da matriz.

1) Observe que esta forma não é muito diferente da abordagem anterior.

2) A principal diferença é que usamos camadas Fully Connected, bem como as camadas Dropout e BatchNormalization.

3) O número de unidades e o número de camadas etc. são os hiperparâmetros aqui como em uma rede neural tradicional.

 #### Observe que usei 50 fatores latentes, pois isso parece fornecer um desempenho razoável. O ajuste mais avançado e a otimização cuidadosa podem fornecer resultados ainda melhores.

In [ ]:
n_latent_factors=50
n_movies=len(df['movieId'].unique())
n_users=len(df['userId'].unique())

In [ ]:
user_input=Input(shape=(1,),name='user_input',dtype='int64')
user_embedding=Embedding(n_users,n_latent_factors,name='user_embedding')(user_input)
user_vec=Flatten(name='FlattenUsers')(user_embedding)
user_vec=Dropout(0.40)(user_vec)

In [ ]:
movie_input=Input(shape=(1,),name='movie_input',dtype='int64')
movie_embedding=Embedding(n_movies,n_latent_factors,name='movie_embedding')(movie_input)
movie_vec=Flatten(name='FlattenMovies')(movie_embedding)
movie_vec=Dropout(0.40)(movie_vec)

In [ ]:
sim=dot([user_vec,movie_vec],name='Simalarity-Dot-Product',axes=1)

## 5.2 ) Especificando a arquitetura do modelo

In [ ]:
nn_inp=Dense(96,activation='relu')(sim)
nn_inp=Dropout(0.4)(nn_inp)
# nn_inp=BatchNormalization()(nn_inp)
nn_inp=Dense(1,activation='relu')(nn_inp)
nn_model =keras.models.Model([user_input, movie_input],nn_inp)
nn_model.summary()


#### Observe o resumo do modelo e também a arquitetura do modelo que você pode ajustar, é claro.

## 5.3 ) Compilando o modelo

In [ ]:
nn_model.compile(optimizer=Adam(lr=1e-3),loss='mse')

In [ ]:
batch_size=128
epochs=20

## 5. 4) Ajuste no conjunto de treinamento e validação no conjunto de validação.

In [ ]:
History = nn_model.fit([train.userId,train.movieId],train.rating, batch_size=batch_size,
                              epochs =epochs, validation_data = ([valid.userId,valid.movieId],valid.rating),
                              verbose = 1)

1. #### Observe que a perda de validação está perto de 0,85, o que é bastante decente. Observe também que diminuiu de 1,26 no caso da Fatoração de Matriz normal para este valor aqui.

#### Da mesma forma, jogando sem nenhum fator latente, outros parâmetros na arquitetura do modelo podem dar resultados ainda melhores !!!!!

In [ ]:
from pylab import rcParams
rcParams['figure.figsize'] = 9, 6
import matplotlib.pyplot as plt
plt.plot(History.history['loss'] , 'g')
plt.plot(History.history['val_loss'] , 'b')
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.grid(True)
plt.show()